In [1]:
# !pip3 install tokenizers

In [2]:
from tokenizers import ByteLevelBPETokenizer
from glob import glob

In [3]:
glob('splitted/*.txt')

['splitted/splitted-watpadd01.txt',
 'splitted/splitted-academia03.txt',
 'splitted/splitted-wiki00.txt',
 'splitted/splitted-academia02.txt',
 'splitted/splitted-parliament00.txt',
 'splitted/splitted-academia04.txt',
 'splitted/splitted-watpadd00.txt',
 'splitted/splitted-news03.txt',
 'splitted/splitted-academia00.txt',
 'splitted/splitted-news00.txt',
 'splitted/splitted-wiki02.txt',
 'splitted/splitted-academia05.txt',
 'splitted/splitted-wiki01.txt',
 'splitted/splitted-academia01.txt',
 'splitted/splitted-news02.txt',
 'splitted/splitted-news01.txt']

In [4]:
tokenizer = ByteLevelBPETokenizer(add_prefix_space=True)

In [5]:
tokenizer.train(
    glob('splitted/*.txt'),
    vocab_size = 50257,
    show_progress = True,
    special_tokens = ['<s>', '<pad>', '</s>', '<|endoftext|>'],
)

In [6]:
tokenizer.save_model('./', 'bahasa')

['./bahasa-vocab.json', './bahasa-merges.txt']

In [7]:
import os
import json
import regex as re
from functools import lru_cache


@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = (
        list(range(ord('!'), ord('~') + 1))
        + list(range(ord('¡'), ord('¬') + 1))
        + list(range(ord('®'), ord('ÿ') + 1))
    )
    cs = bs[:]
    n = 0
    for b in range(2 ** 8):
        if b not in bs:
            bs.append(b)
            cs.append(2 ** 8 + n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))


def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


class Encoder:
    def __init__(self, encoder, bpe_merges, errors='replace'):
        self.encoder = encoder
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.errors = errors  # how to handle errors in decoding
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}

        # Should haved added re.IGNORECASE so BPE merges can happen for
        # capitalized versions of contractions
        self.pat = re.compile(
            r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        )

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(
                pairs, key=lambda pair: self.bpe_ranks.get(pair, float('inf'))
            )
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except BaseException:
                    new_word.extend(word[i:])
                    break

                if (
                    word[i] == first
                    and i < len(word) - 1
                    and word[i + 1] == second
                ):
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(
                self.encoder[bpe_token]
                for bpe_token in self.bpe(token).split(' ')
            )
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode(
            'utf-8', errors=self.errors
        )
        return text


In [8]:
with open('bahasa-vocab.json', 'r') as f:
    en = json.load(f)
with open('bahasa-merges.txt', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
    
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
enc_malay = Encoder(
    encoder=en,
    bpe_merges=bpe_merges,
)

In [9]:
enc_malay.encode('saya')

[22808]

In [10]:
enc_malay.encode('saya suka makan')

[22808, 3147, 1243]